In [2]:
from pyscf import gto, scf, mp, cc

a = 2 # bond length in a cluster
d = 3 # distance between each cluster
unit = 'b' # unit of length
na = 3  # size of a cluster (monomer)
nc = 2 # set as integer multiple of monomers
spin = 1 # spin per monomer
frozen = 0 # frozen orbital per monomer
elmt = 'H'
basis = 'sto6g'
atoms = ""
for n in range(nc*na):
    shift = ((n - n % na) // na) * (d-a)
    atoms += f"{elmt} {n*a+shift:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis=basis, spin=2, unit=unit, verbose=4, max_memory=16000)
mf = scf.UHF(mol).density_fit()
mf.kernel()

nfrozen = 0
# canonical
# mmp = mp.UMP2(mf, frozen=nfrozen)
# mmp.kernel()

# mycc = cc.UCCSD(mf, frozen=nfrozen)
# eris = mycc.ao2mo()
# mycc.kernel(eris=eris)
# et = mycc.ccsd_t(eris=eris)

System: uname_result(system='Linux', node='sharmagroup-rn', release='6.14.0-36-generic', version='#36~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Wed Oct 15 15:45:17 UTC 2', machine='x86_64')  Threads 16
Python 3.11.14 (main, Oct 21 2025, 18:31:21) [GCC 11.2.0]
numpy 2.3.1  scipy 1.16.2  h5py 3.14.0
Date: Mon Dec  8 14:39:40 2025
PySCF version 2.11.0
PySCF path  /home/sharmagroup/sharmagroup/pyscf
GIT HEAD (branch master) 3d1768f5e33b144b606c3d2c81c12ee54d794501

[ENV] PYSCF_EXT_PATH /home/sharmagroup/sharmagroup/pyscf-forge
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 6
[INPUT] num. electrons = 6
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 2
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = b
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr  

In [3]:
from pyscf import lo
import numpy as np
orbocca = mf.mo_coeff[0][:,nfrozen:np.count_nonzero(mf.mo_occ[0])]
orbloca = lo.PipekMezey(mol, orbocca).kernel()
orboccb = mf.mo_coeff[1][:,nfrozen:np.count_nonzero(mf.mo_occ[1])]
orblocb = lo.PipekMezey(mol, orboccb).kernel()
lo_coeff = [orbloca, orblocb]

oa = [[[i],[]] for i in range(orbloca.shape[1])]
ob = [[[],[i]] for i in range(orblocb.shape[1])]
frag_lolist = oa + ob



******** <class 'pyscf.lo.pipek.PipekMezey'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.05
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
pop_method = meta_lowdin
Set conv_tol_grad to 0.000316228
macro= 1  f(x)= 2.7484458841607  delta_f= 2.74845  |g|= 0.0661674  1 KF 2 Hx
macro= 2  f(x)= 2.7484458841646  delta_f= 3.8578e-12  |g|= 1.25888e-05  1 KF 1 Hx
macro X = 2  f(x)= 2.7484458841646  |g|= 1.25888e-05  4 intor 2 KF 3 Hx


******** <class 'pyscf.lo.pipek.PipekMezey'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.05
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
pop_method = meta_lowdin


In [5]:
from collections.abc import Iterable
from pyscf.lno import ulnoccsd
from ad_afqmc.lno_afqmc import ulno_afqmc

thresh = 1e-4
run_frg_list = [0]
chol_cut = 1e-5

options = {'n_eql': 5,
        'n_prop_steps': 50,
        'n_ene_blocks': 1,
        'n_sr_blocks': 10,
        'n_blocks': 10,
        'n_walkers': 1,
        'seed': 98,
        'walker_type': 'uhf',
        'trial': 'uccsd_pt_ad',
        'dt':0.005,
        'free_projection':False,
        'ad_mode':None,
        'use_gpu': False,
        'max_error': 1e-4
        }


mlno = ulnoccsd.ULNOCCSD_T(mf, lo_coeff, frag_lolist, frozen=nfrozen).set(verbose=4)
mlno.lno_thresh = [thresh*10,thresh]
lno_thresh = mlno.lno_thresh
lno_type = ['1h','1h']
lno_thresh = [1e-5, 1e-6] if lno_thresh is None else lno_thresh
print(lno_thresh)
lno_pct_occ = None
lno_norb = None
lo_proj_thresh = 1e-10
lo_proj_thresh_active = 0.1
eris = None

if run_frg_list is not None:
    frag_lolist = [frag_lolist[i] for i in run_frg_list]

nfrag = len(frag_lolist)
if lno_pct_occ is None:
    lno_pct_occ = [None, None]
if lno_norb is None:
    lno_norb = [[None,None]] * nfrag
mf = mlno._scf

if eris is None: eris = mlno.ao2mo()

# Loop over fragment
# frag_res = [None] * nfrag
for ifrag, loidx in enumerate(frag_lolist):
    if len(loidx) == 2 and isinstance(loidx[0], Iterable): # Unrestricted
        orbloc = [lo_coeff[0][:,loidx[0]], lo_coeff[1][:,loidx[1]]]
        lno_param = [
            [
                {
                    'thresh': (
                        lno_thresh[i][s] if isinstance(lno_thresh[i], Iterable)
                        else lno_thresh[i]
                    ),
                    'pct_occ': (
                        lno_pct_occ[i][s] if isinstance(lno_pct_occ[i], Iterable)
                        else lno_pct_occ[i]
                    ),
                    'norb': (
                        lno_norb[ifrag][i][s] if isinstance(lno_norb[ifrag][i], Iterable)
                        else lno_norb[ifrag][i]
                    ),
                } for i in [0, 1]
            ] for s in range(2)
        ]

    else:
        orbloc = lo_coeff[:,loidx]
        lno_param = [{'thresh': lno_thresh[i], 'pct_occ': lno_pct_occ[i],
                        'norb': lno_norb[ifrag][i]} for i in [0,1]]

    lno_coeff, frozen, uocc_loc, frag_msg = mlno.make_las(eris, orbloc, lno_type, lno_param)
    # frag_res[ifrag], frag_msg = mlno.impurity_solve(mf, mo_coeff, uocc_loc, eris, frozen=frozen)
        
        # return (emp2,eccsd,ept)
        # run_afqmc(options,nproc)
        # os.system(f'mv afqmc.out lnoafqmc.out{ifrag+1}')


[0.001, 0.0001]
LO occ proj: 1 active | 0 standby | 3 orthogonal
LO occ proj: 0 active | 0 standby | 2 orthogonal


In [6]:
frozen

[array([0, 1, 2]), array([0, 3, 4, 5])]

In [7]:
mol = mf.mol
nocc_a = int(sum(mf.mo_occ[0]))
actfrag_a = np.array([i for i in range(mol.nao) if i not in frozen[0]])
frzocc_a = np.array([i for i in range(nocc_a) if i in frozen[0]])
actocc_a = np.array([i for i in range(nocc_a) if i in actfrag_a])
actvir_a = np.array([i for i in range(nocc_a,mol.nao) if i in actfrag_a])
nfrzocc_a = len(frzocc_a)
nactocc_a = len(actocc_a)
nactvir_a = len(actvir_a)
nactorb_a = len(actfrag_a)
nocc_b = int(sum(mf.mo_occ[1]))
actfrag_b = np.array([i for i in range(mol.nao) if i not in frozen[1]])
frzocc_b = np.array([i for i in range(nocc_b) if i in frozen[1]])
actocc_b = np.array([i for i in range(nocc_b) if i in actfrag_b])
actvir_b = np.array([i for i in range(nocc_b,mol.nao) if i in actfrag_b])
nfrzocc_b = len(frzocc_b)
nactocc_b = len(actocc_b)
nactvir_b = len(actvir_b)
nactorb_b = len(actfrag_b)

ncas = (nactorb_a, nactorb_b)
ncore = (nfrzocc_a, nfrzocc_b)
nelec = (nactocc_a, nactocc_b)
print(ncas)
print(ncore)
print(nelec)

(3, 2)
(3, 1)
(1, 1)


In [8]:
mo_occ = mlno.mo_occ
frozen, maskact = ulnoccsd.get_maskact(frozen, [mo_occ[0].size, mo_occ[1].size])
mcc = ulnoccsd.UCCSD(mf, mo_coeff=lno_coeff, frozen=frozen).set(verbose=mlno.verbose_imp)
mcc._s1e = mlno._s1e
mcc._h1e = mlno._h1e
mcc._vhf = mlno._vhf
if mlno.kwargs_imp is not None:
    mcc = mcc.set(**mlno.kwargs_imp)

(emp2,eccsd,ept), t1, t2, prjlo =\
    ulno_afqmc.ulno_ccsd(mcc, lno_coeff, uocc_loc, mo_occ, maskact, ccsd_t=True)
print(emp2,eccsd,ept)


WARN: CCSD detected DF being used in the HF object. MO integrals are computed based on the DF 3-index tensors.
It's recommended to use dfccsd.CCSD for the DF-CCSD calculations

# LNO-UCCSD (T1 = 0) fragment energy: -0.00828354447785702
-0.0036944627461361203 -0.008423497276034005 0.0


In [9]:
options = {'n_eql': 5,
        'n_prop_steps': 50,
        'n_ene_blocks': 1,
        'n_sr_blocks': 10,
        'n_blocks': 10,
        'n_walkers': 1,
        'seed': 98,
        'walker_type': 'uhf',
        'trial': 'uccsd_pt_ad',
        'dt':0.005,
        'free_projection':False,
        'ad_mode':None,
        'use_gpu': False,
        'max_error': 1e-4
        }

from mpi4py import MPI
if not MPI.Is_finalized():
    MPI.Finalize()

nelec, ncas = ulno_afqmc.prep_afqmc(
    mf,lno_coeff,t1,t2,frozen,prjlo,
    options,chol_cut=chol_cut)

print(nelec, ncas)

# Calculating Effective Active Space One-electron Integrals
# Generating Cholesky Integrals
# Constracting common LAS that span both Alpha and Beta active space
# Active space loss: (1.10e+00, 8.66e-01)
# Active space loss: (8.47e-01, 8.88e-01)
# Active space loss: (3.91e-01, 9.11e-01)
# Active space loss: (3.91e-01, 3.02e-01)
# Active space loss: (4.60e-02, 4.83e-02)
# Minimum number of SVD orbitals to span both Alpha and Beta active space: 5
# Composing AO ERIs from DF basis
# Composing active space MO ERIs from AO ERIs
# Decomposing MO ERIs to Cholesky vectors
# Cholesky cutoff is: 1e-05
# Finished calculating Cholesky integrals
# Size of the correlation space
# Number of electrons: (1, 1)
# Number of basis functions: (3, 2)
# Alpha Basis Cholesky shape: (10, 3, 3)
#  Beta Basis Cholesky shape: (10, 2, 2)
(1, 1) (3, 2)


In [1]:
from ad_afqmc import config
config.setup_jax()
MPI = config.setup_comm()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
from ad_afqmc.lno_afqmc import ulno_afqmc
ham_data, prop, trial, wave_data, sampler, options, _ = (
    ulno_afqmc._prep_afqmc())

# Hostname: sharmagroup-rn
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Hostname: sharmagroup-rn
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: (3, 2)
# nelec: (1, 1)
#
# n_eql: 5
# n_prop_steps: 50
# n_ene_blocks: 1
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 1
# seed: 98
# walker_type: uhf
# trial: uccsd_pt_ad
# dt: 0.005
# free_projection: False
# use_gpu: False
# max_error: 0.0001
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
#


In [2]:
import time
from jax import numpy as jnp
from jax import random
init_time = time.time()
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

### initialize propagation
seed = options["seed"]
init_walkers = None
# dm_up = jnp.array(wave_data["mo_coeff"][0] @ wave_data["mo_coeff"][0].T.conj())
# dm_dn = jnp.array(wave_data["mo_coeff"][1] @ wave_data["mo_coeff"][1].T.conj())
# trial_rdm1 = [dm_up, dm_dn]
trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1
ham_data = trial._build_measurement_intermediates(ham_data, wave_data)
ham_data = prop._build_propagation_intermediates(ham_data, trial, wave_data)

prop_data = prop.init_prop_data(trial, wave_data, ham_data, init_walkers)
if jnp.abs(jnp.sum(prop_data["overlaps"])) < 1.0e-6:
    raise ValueError(
        "Initial overlaps are zero. Pass walkers with non-zero overlap."
    )
prop_data["key"] = random.PRNGKey(seed + rank)

prop_data["overlaps"] = trial.calc_overlap(prop_data["walkers"], wave_data)
prop_data["n_killed_walkers"] = 0
prop_data["pop_control_ene_shift"] = prop_data["e_estimate"]
print(prop_data["e_estimate"])


-3.0709389011315706


In [ ]:
# import opt_einsum as oe
# from jax import vmap
# def _calc_ecorr(trial, walker_up, walker_dn, ham_data, wave_data):
#     '''uhf correlation energy'''
#     # <HF|H-E0|walker>/<HF|walker>
#     nocca, noccb = trial.nelec
#     # rot_h1a, rot_h1b = ham_data['rot_h1']
#     rot_chola, rot_cholb = ham_data['rot_chol']
#     GFa, GFb = trial._calc_green(walker_up, walker_dn, wave_data)
#     lga = oe.contract('gij,jk->gik', rot_chola[:,:nocca,nocca:],
#                     GFa.T[nocca:,:nocca], backend="jax")
#     lgb = oe.contract('gij,jk->gik', rot_cholb[:,:noccb,noccb:],
#                     GFb.T[noccb:,:noccb], backend="jax")
#     tr_lga = oe.contract('gii->g',lga)
#     tr_lgb = oe.contract('gii->g',lgb)
#     lglg_aa = oe.contract('g,g->',tr_lga,tr_lga) - oe.contract('gij,gji->',lga,lga)
#     lglg_ab = oe.contract('g,g->',tr_lga,tr_lgb)*2
#     lglg_bb = oe.contract('g,g->',tr_lgb,tr_lgb) - oe.contract('gij,gji->',lgb,lgb)
#     ecorr = 0.5*(lglg_aa + lglg_ab + lglg_bb)
#     return jnp.real(ecorr)

# def _calc_eorb(trial, walker_up, walker_dn, ham_data, wave_data):
#     '''uhf correlation energy'''
#     # <HF|H-E0|walker>/<HF|walker>
#     nocca, noccb = trial.nelec
#     # rot_h1a, rot_h1b = ham_data['rot_h1']
#     prjloa, prjlob = wave_data['prjlo']
#     rot_chola, rot_cholb = ham_data['rot_chol']
#     GFa, GFb = trial._calc_green(walker_up, walker_dn, wave_data)
#     lga = oe.contract('gij,jk->gik', rot_chola[:,:nocca,nocca:],
#                     GFa.T[nocca:,:nocca], backend="jax")
#     lgb = oe.contract('gij,jk->gik', rot_cholb[:,:noccb,noccb:],
#                     GFb.T[noccb:,:noccb], backend="jax")
#     tr_lga = oe.contract('gii->g',lga)
#     tr_lgb = oe.contract('gii->g',lgb)
#     lga_orb = oe.contract('gik,ik->g',lga,prjloa)
#     lgb_orb = oe.contract('gik,ik->g',lgb,prjlob)
#     lglg_aa = oe.contract('g,g->',lga_orb,tr_lga) - oe.contract('gij,gjk,ik->',lga,lga,prjloa)
#     lglg_ab = oe.contract('g,g->',lga_orb,tr_lgb) + oe.contract('g,g->',tr_lga,lgb_orb)
#     lglg_bb = oe.contract('g,g->',lgb_orb,tr_lgb) - oe.contract('gij,gjk,ik->',lgb,lgb,prjlob)
#     eorb = 0.5*(lglg_aa + lglg_ab + lglg_bb)
#     return jnp.real(eorb)

In [ ]:
# import jax
# from jax import lax, jvp, vmap
# import opt_einsum as oe
# def _t_orb(trial, walker_up, walker_dn, wave_data: dict) -> complex:
#     '''
#     <HF|t1+t2|walker>_i 
#     = (C_ia <HF|i+ a|walker>/<HF|walker> + C_iajb <HF|i+ j+ a b|walker>/<HF|walker>) * <HF|walker>
#     = (C_ia G_ia + C_iajb (G_ia G_jb-G_ib G_ja)) * <HF|walker>
#     prj onto spin-orbit i
#     '''
#     pa, pb = wave_data["prjlo"]
#     nocca, noccb = trial.nelec
#     t1a, t1b = wave_data["t1a"], wave_data["t1b"]
#     t2aa, t2ab, t2bb = wave_data["t2aa"], wave_data["t2ab"], wave_data["t2bb"]
#     greena, greenb = trial._calc_green(walker_up, walker_dn, wave_data)
#     greena, greenb = greena[:nocca, nocca:], greenb[:noccb, noccb:]
#     o0 = trial._calc_overlap(walker_up,walker_dn,wave_data)
#     o1 = oe.contract("ia,ka,ik->", t1a, greena, pa, backend="jax") \
#             + oe.contract("ia,ka,ik->", t1b, greenb, pa, backend="jax")
#     o2 = (oe.contract("iajb,ka,jb,ik->", t2aa, greena, greena, pa, backend="jax")
#             + oe.contract("iajb,ka,jb,ik->", t2bb, greenb, greenb, pb, backend="jax")
#             + oe.contract("iajb,ka,jb,ik->", t2ab, greena, greenb, pa, backend="jax")
#             + oe.contract("jbia,jb,ka,ik->", t2ab, greena, greenb, pb, backend="jax")) * 0.5
#     return (o1 + o2) * o0

In [ ]:
# import jax
# from jax import lax, jvp, vmap
# import opt_einsum as oe

# def _calc_eorb_pt(
#     trial,
#     walker_up: jax.Array,
#     walker_dn: jax.Array,
#     ham_data: dict,
#     wave_data: dict,
# ) -> complex:
#     norba, norbb = trial.norb
#     nocca, noccb = trial.nelec
#     h0 = ham_data["h0"]
#     prja, prjb = wave_data["prjlo"]
#     t1a, t1b = wave_data["t1a"], wave_data["t1b"]
#     t2aa, t2ab, t2bb = wave_data["t2aa"], wave_data["t2ab"], wave_data["t2bb"]
#     chola, cholb = ham_data["chol"]
#     chola = chola.reshape(-1, norba, norba)
#     cholb = cholb.reshape(-1, norbb, norbb)
#     # chol = [chola, cholb]
#     # h1_mod = ham_data['h1_mod']
#     greena = (walker_up.dot(jnp.linalg.inv(walker_up[:nocca, :]))).T
#     greenb = (walker_dn.dot(jnp.linalg.inv(walker_dn[:noccb, :]))).T
#     green_occa = greena[:, nocca:].copy()
#     green_occb = greenb[:, noccb:].copy()
#     greenpa = jnp.vstack((green_occa, -jnp.eye(norba-nocca)))
#     greenpb = jnp.vstack((green_occb, -jnp.eye(norbb-noccb)))

#     # chola = ham_data["chol"][0].reshape(-1, self.norb, self.norb)
#     # cholb = ham_data["chol"][1].reshape(-1, self.norb, self.norb)
#     rot_chola = chola[:, :nocca, :]
#     rot_cholb = cholb[:, :noccb, :]
#     h1a = ham_data["h1"][0]
#     h1b = ham_data["h1"][1]
#     hga = oe.contract("pj,pj->", h1a[:nocca, :], greena, backend="jax")
#     hgb = oe.contract("pj,pj->", h1b[:noccb, :], greenb, backend="jax")
#     e1_0 = hga + hgb # <HF|h1|walker>/<HF|walker>
#     # hg = hga + hgb


#     # 1 body energy
#     # ref
#     # e1_0 = hg # <HF|h1|walker>/<HF|walker>

#     # single excitations = t_ia (G_ia G_pq - G_iq Gp_pa) h_pq
#     # t1ga = oe.contract("ia,ia->", t1a, green_occa, backend="jax")
#     # t1gb = oe.contract("ia,ia->", t1b, green_occb, backend="jax")
#     t1ga = oe.contract("ia,ik,ka->", t1a, prja, green_occa, backend="jax")
#     t1gb = oe.contract("ia,ik,ka->", t1b, prjb, green_occb, backend="jax")
#     # t1g = t1ga + t1gb
#     t1g = t1ga + t1gb # <HF|T1|walker>/<HF|walker>
#     e1_1_1 = t1g * e1_0
#     # gpt1_a = greenpa @ t1a.T
#     # gpt1_b = greenpb @ t1b.T
#     # gpt1a = oe.contract("pa,ai->pi", greenpa, t1a.T, backend="jax")
#     # gpt1b = oe.contract("pa,ai->pi", greenpb, t1b.T, backend="jax")
#     # t1_green_a = gpt1a @ greena
#     # t1_green_b = gpt1b @ greenb
#     gh1gpa = oe.contract("iq,pq,pa->ia", greena, h1a, greenpa, backend="jax")
#     gh1gpb = oe.contract("iq,pq,pa->ia", greenb, h1b, greenpb, backend="jax")
#     e1_1_2 = -(oe.contract("ia,ik,ka->", t1a, prja, gh1gpa, backend="jax")
#                + oe.contract("ia,ik,ka->", t1b, prjb, gh1gpb, backend="jax"))
#     e1_1 = e1_1_1 + e1_1_2 # <HF|T1 h1|walker>/<HF|walker>

#     # double excitations
#     # t2g_a = oe.contract("ptqu,pt->qu", t2_aa, green_occ_a, backend="jax") / 4
#     # t2g_b = oe.contract("ptqu,pt->qu", t2_bb, green_occ_b, backend="jax") / 4
#     # t2g_ab_a = oe.contract("ptqu,qu->pt", t2_ab, green_occ_b, backend="jax")
#     # t2g_ab_b = oe.contract("ptqu,pt->qu", t2_ab, green_occ_a, backend="jax")
#     # gt2g_a = oe.contract("qu,qu->", t2g_a, green_occ_a, backend="jax")
#     # gt2g_b = oe.contract("qu,qu->", t2g_b, green_occ_b, backend="jax")
#     # gt2g_ab = oe.contract("pt,pt->", t2g_ab_a, green_occ_a, backend="jax")
#     # gt2g = 2 * (gt2g_a + gt2g_b) + gt2g_ab
#     # e1_2_1 = hg * gt2g
#     # t2_green_a = (greenp_a @ t2g_a.T) @ green_a
#     # t2_green_ab_a = (greenp_a @ t2g_ab_a.T) @ green_a
#     # t2_green_b = (greenp_b @ t2g_b.T) @ green_b
#     # t2_green_ab_b = (greenp_b @ t2g_ab_b.T) @ green_b
#     # gggh1a = oe.contract("ia,jb,pq,pq->", green_occa, greena, greena, h1a[:nocca, :], backend="jax")
#     # gggh1b = oe.contract("ia,jb,pq,pq->", green_occb, greenb, greenb, h1a[:noccb, :], backend="jax")
#     h1gpa = oe.contract("pq,pa->qa", h1a, greenpa, backend="jax")
#     h1gpb = oe.contract("pq,pa->qa", h1b, greenpb, backend="jax")
#     e1_2_2_a = -oe.contract(
#         "ij,ij->", h1_a, 4 * t2_green_a + t2_green_ab_a, backend="jax"
#     )
#     e1_2_2_b = -oe.contract(
#         "ij,ij->", h1_b, 4 * t2_green_b + t2_green_ab_b, backend="jax"
#     )
#     e1_2_2 = e1_2_2_a + e1_2_2_b
#     e1_2 = e1_2_1 + e1_2_2 # <HF|T2 h1|walker>/<HF|walker>

#     # two body energy
#     # ref
#     lg_a = oe.contract("gpj,pj->g", rot_chol_a, green_a, backend="jax")
#     lg_b = oe.contract("gpj,pj->g", rot_chol_b, green_b, backend="jax")
#     e2_0_1 = ((lg_a + lg_b) @ (lg_a + lg_b)) / 2.0
#     lg1_a = oe.contract("gpj,qj->gpq", rot_chol_a, green_a, backend="jax")
#     lg1_b = oe.contract("gpj,qj->gpq", rot_chol_b, green_b, backend="jax")
#     e2_0_2 = (
#         -(
#             jnp.sum(vmap(lambda x: x * x.T)(lg1_a))
#             + jnp.sum(vmap(lambda x: x * x.T)(lg1_b))
#         )
#         / 2.0
#     )
#     e2_0 = e2_0_1 + e2_0_2 # <HF|h2|walker>/<HF|walker>

#     # single excitations
#     e2_1_1 = e2_0 * t1g
#     lt1g_a = oe.contract("gij,ij->g", chol_a, t1_green_a, backend="jax")
#     lt1g_b = oe.contract("gij,ij->g", chol_b, t1_green_b, backend="jax")
#     e2_1_2 = -((lt1g_a + lt1g_b) @ (lg_a + lg_b))
#     t1g1_a = t1_a @ green_occ_a.T
#     t1g1_b = t1_b @ green_occ_b.T
#     e2_1_3_1 = oe.contract(
#         "gpq,gqr,rp->", lg1_a, lg1_a, t1g1_a, backend="jax"
#     ) + oe.contract("gpq,gqr,rp->", lg1_b, lg1_b, t1g1_b, backend="jax")
#     lt1g_a = oe.contract(
#         "gip,qi->gpq", ham_data["lt1_a"], green_a, backend="jax"
#     )
#     lt1g_b = oe.contract(
#         "gip,qi->gpq", ham_data["lt1_b"], green_b, backend="jax"
#     )
#     e2_1_3_2 = -oe.contract(
#         "gpq,gqp->", lt1g_a, lg1_a, backend="jax"
#     ) - oe.contract("gpq,gqp->", lt1g_b, lg1_b, backend="jax")
#     e2_1_3 = e2_1_3_1 + e2_1_3_2
#     e2_1 = e2_1_1 + e2_1_2 + e2_1_3 # <HF|T1 h2|walker>/<HF|walker>

#     # double excitations
#     e2_2_1 = e2_0 * gt2g
#     lt2g_a = oe.contract(
#         "gij,ij->g",
#         chol_a,
#         8 * t2_green_a + 2 * t2_green_ab_a,
#         backend="jax",
#     )
#     lt2g_b = oe.contract(
#         "gij,ij->g",
#         chol_b,
#         8 * t2_green_b + 2 * t2_green_ab_b,
#         backend="jax",
#     )
#     e2_2_2_1 = -((lt2g_a + lt2g_b) @ (lg_a + lg_b)) / 2.0

#     def scanned_fun(carry, x):
#         chol_a_i, rot_chol_a_i, chol_b_i, rot_chol_b_i = x
#         gl_a_i = oe.contract("pj,ji->pi", green_a, chol_a_i, backend="jax")
#         gl_b_i = oe.contract("pj,ji->pi", green_b, chol_b_i, backend="jax")
#         lt2_green_a_i = oe.contract(
#             "pi,ji->pj",
#             rot_chol_a_i,
#             8 * t2_green_a + 2 * t2_green_ab_a,
#             backend="jax",
#         )
#         lt2_green_b_i = oe.contract(
#             "pi,ji->pj",
#             rot_chol_b_i,
#             8 * t2_green_b + 2 * t2_green_ab_b,
#             backend="jax",
#         )
#         carry[0] += 0.5 * (
#             oe.contract("pi,pi->", gl_a_i, lt2_green_a_i, backend="jax")
#             + oe.contract("pi,pi->", gl_b_i, lt2_green_b_i, backend="jax")
#         )
#         glgp_a_i = oe.contract(
#             "pi,it->pt", gl_a_i, greenp_a, backend="jax"
#         )
#         glgp_b_i = oe.contract(
#             "pi,it->pt", gl_b_i, greenp_b, backend="jax"
#         )
#         l2t2_a = 0.5 * oe.contract(
#             "pt,qu,ptqu->",
#             glgp_a_i,
#             glgp_a_i,
#             t2_aa,
#             backend="jax",
#         )
#         l2t2_b = 0.5 * oe.contract(
#             "pt,qu,ptqu->",
#             glgp_b_i,
#             glgp_b_i,
#             t2_bb,
#             backend="jax",
#         )
#         l2t2_ab = oe.contract(
#             "pt,qu,ptqu->",
#             glgp_a_i,
#             glgp_b_i,
#             t2_ab,
#             backend="jax",
#         )
#         carry[1] += l2t2_a + l2t2_b + l2t2_ab
#         return carry, 0.0

#     [e2_2_2_2, e2_2_3], _ = lax.scan(
#         scanned_fun, [0.0, 0.0], (chol_a, rot_chol_a, chol_b, rot_chol_b)
#     )
#     e2_2_2 = e2_2_2_1 + e2_2_2_2
#     e2_2 = e2_2_1 + e2_2_2 + e2_2_3 # <HF|T2 h2|walker>/<HF|walker>

#     t = t1g + gt2g # <HF|T1+T2|walker>/<HF|walker>
#     e0 = h0 + e1_0 + e2_0 # h0 + <HF|h1+h2|walker>/<HF|walker>
#     e1 = e1_1 + e1_2 + e2_1 + e2_2 # <HF|(T1+T2)(h1+h2)|walker>/<HF|walker>

#     return jnp.real(t), jnp.real(e0), jnp.real(e1)

In [5]:
import opt_einsum as oe
prja, prjb = wave_data['prjlo']
t1a, t1b = wave_data["t1a"], wave_data["t1b"]
t2aa, t2ab, t2bb = wave_data["t2aa"], wave_data["t2ab"], wave_data["t2bb"]
t1a_prj = oe.contract('ia,ik->ka',t1a,prja,backend='jax')
t1b_prj = oe.contract('ia,ik->ka',t1b,prjb,backend='jax')
t2aa_prj = oe.contract('iajb,ik->kajb',t2aa,prja,backend='jax')
t2ab_prj = oe.contract('iajb,ik->kajb',t2ab,prja,backend='jax')
t2ba_prj = oe.contract('jbia,ik->kajb',t2ab,prjb,backend='jax')
t2bb_prj = oe.contract('iajb,ik->kajb',t2bb,prjb,backend='jax')
wave_data["t1a_prj"], wave_data["t1b_prj"] = t1a_prj, t1b_prj
wave_data["t2aa_prj"], wave_data["t2ab_prj"] = t2aa_prj, t2ab_prj
wave_data["t2ba_prj"], wave_data["t2bb_prj"] = t2ba_prj, t2bb_prj

In [ ]:
import jax
from jax import lax, jvp, vmap
import opt_einsum as oe

def _calc_energy_pt(
    trial,
    walker_up: jax.Array,
    walker_dn: jax.Array,
    ham_data: dict,
    wave_data: dict,
) -> complex:
    norb_a, norb_b = trial.norb
    nocc_a, nocc+b = trial.nelec
    h0 = ham_data["h0"]
    t1_a, t1_b = wave_data["t1a_prj"], wave_data["t1b_prj"]
    t2_aa, t2_ab = wave_data["t2aa_prj"], wave_data["t2ab_prj"],
    t2_ba, t2_bb = wave_data["t2ba_prj"], wave_data["t2bb_prj"]
    chol_a, chol_b = ham_data["chol"]
    chol_a = chol_a.reshape(-1, norb_a, norb_a)
    chol_b = chol_b.reshape(-1, norb_b, norb_b)

    green_a = (walker_up.dot(jnp.linalg.inv(walker_up[:nocc_a, :]))).T
    green_b = (walker_dn.dot(jnp.linalg.inv(walker_dn[:nocc_b, :]))).T
    green_occ_a = green_a[:, nocc_a:].copy()
    green_occ_b = green_b[:, nocc_b:].copy()
    greenp_a = jnp.vstack((green_occ_a, -jnp.eye(norb_a - nocc_a)))
    greenp_b = jnp.vstack((green_occ_b, -jnp.eye(norb_b - nocc_b)))

    rot_chol_a = chol_a[:, :nocc_a, :]
    rot_chol_b = chol_b[:, :nocc_b, :]
    h1_a = ham_data["h1"][0]
    h1_b = ham_data["h1"][1]
    hg_a = oe.contract("pj,pj->", h1_a[:nocc_a, :], green_a, backend="jax")
    hg_b = oe.contract("pj,pj->", h1_b[:nocc_b, :], green_b, backend="jax")
    hg = hg_a + hg_b

    # 0 body energy
    h0 = ham_data["h0"]

    # 1 body energy
    # ref
    e1_0 = hg # <HF|h1|walker>/<HF|walker>

    # single excitations
    t1g_a = oe.contract("ia,ia->", t1_a, green_occ_a, backend="jax")
    t1g_b = oe.contract("ia,ia->", t1_b, green_occ_b, backend="jax")
    t1g = t1g_a + t1g_b
    e1_1_1 = t1g * hg
    gpt1_a = greenp_a @ t1_a.T
    gpt1_b = greenp_b @ t1_b.T
    t1_green_a = gpt1_a @ green_a
    t1_green_b = gpt1_b @ green_b
    e1_1_2 = -(
        oe.contract("pq,pq->", h1_a, t1_green_a, backend="jax")
        + oe.contract("pq,pq->", h1_b, t1_green_b, backend="jax")
    )
    e1_1 = e1_1_1 + e1_1_2 # <HF|T1 h1|walker>/<HF|walker>

    # double excitations
    t2g_a = oe.contract("iajb,ia->jb", t2_aa, green_occ_a, backend="jax") / 4
    t2g_b = oe.contract("iajb,ia->jb", t2_bb, green_occ_b, backend="jax") / 4
    t2g_ab_a = oe.contract("iajb,jb->ia", t2_ab, green_occ_b, backend="jax") / 2
    t2g_ba_a = oe.contract("iajb,ia->jb", t2_ba, green_occ_b, backend="jax") / 2
    t2g_ab_b = oe.contract("iajb,ia->jb", t2_ab, green_occ_a, backend="jax") / 2
    gt2g_a = oe.contract("jb,jb->", t2g_a, green_occ_a, backend="jax")
    gt2g_b = oe.contract("jb,jb->", t2g_b, green_occ_b, backend="jax")
    gt2g_ab = oe.contract("ia,ia->", t2g_ab_a, green_occ_a, backend="jax")
    gt2g = 2 * (gt2g_a + gt2g_b) + gt2g_ab
    e1_2_1 = hg * gt2g
    t2_green_a = (greenp_a @ t2g_a.T) @ green_a
    t2_green_ab_a = (greenp_a @ t2g_ab_a.T) @ green_a
    t2_green_b = (greenp_b @ t2g_b.T) @ green_b
    t2_green_ab_b = (greenp_b @ t2g_ab_b.T) @ green_b
    e1_2_2_a = -oe.contract(
        "ij,ij->", h1_a, 4 * t2_green_a + t2_green_ab_a, backend="jax"
    )
    e1_2_2_b = -oe.contract(
        "ij,ij->", h1_b, 4 * t2_green_b + t2_green_ab_b, backend="jax"
    )
    e1_2_2 = e1_2_2_a + e1_2_2_b
    e1_2 = e1_2_1 + e1_2_2 # <HF|T2 h1|walker>/<HF|walker>

    # two body energy
    # ref
    lg_a = oe.contract("gpj,pj->g", rot_chol_a, green_a, backend="jax")
    lg_b = oe.contract("gpj,pj->g", rot_chol_b, green_b, backend="jax")
    e2_0_1 = ((lg_a + lg_b) @ (lg_a + lg_b)) / 2.0
    lg1_a = oe.contract("gpj,qj->gpq", rot_chol_a, green_a, backend="jax")
    lg1_b = oe.contract("gpj,qj->gpq", rot_chol_b, green_b, backend="jax")
    e2_0_2 = (
        -(
            jnp.sum(vmap(lambda x: x * x.T)(lg1_a))
            + jnp.sum(vmap(lambda x: x * x.T)(lg1_b))
        )
        / 2.0
    )
    e2_0 = e2_0_1 + e2_0_2 # <HF|h2|walker>/<HF|walker>

    # single excitations
    e2_1_1 = e2_0 * t1g
    lt1g_a = oe.contract("gij,ij->g", chol_a, t1_green_a, backend="jax")
    lt1g_b = oe.contract("gij,ij->g", chol_b, t1_green_b, backend="jax")
    e2_1_2 = -((lt1g_a + lt1g_b) @ (lg_a + lg_b))
    t1g1_a = t1_a @ green_occ_a.T
    t1g1_b = t1_b @ green_occ_b.T
    e2_1_3_1 = oe.contract(
        "gpq,gqr,rp->", lg1_a, lg1_a, t1g1_a, backend="jax"
    ) + oe.contract("gpq,gqr,rp->", lg1_b, lg1_b, t1g1_b, backend="jax")
    lt1g_a = oe.contract(
        "gip,qi->gpq", ham_data["lt1_a"], green_a, backend="jax"
    )
    lt1g_b = oe.contract(
        "gip,qi->gpq", ham_data["lt1_b"], green_b, backend="jax"
    )
    e2_1_3_2 = -oe.contract(
        "gpq,gqp->", lt1g_a, lg1_a, backend="jax"
    ) - oe.contract("gpq,gqp->", lt1g_b, lg1_b, backend="jax")
    e2_1_3 = e2_1_3_1 + e2_1_3_2
    e2_1 = e2_1_1 + e2_1_2 + e2_1_3 # <HF|T1 h2|walker>/<HF|walker>

    # double excitations
    e2_2_1 = e2_0 * gt2g
    lt2g_a = oe.contract(
        "gij,ij->g",
        chol_a,
        8 * t2_green_a + 2 * t2_green_ab_a,
        backend="jax",
    )
    lt2g_b = oe.contract(
        "gij,ij->g",
        chol_b,
        8 * t2_green_b + 2 * t2_green_ab_b,
        backend="jax",
    )
    e2_2_2_1 = -((lt2g_a + lt2g_b) @ (lg_a + lg_b)) / 2.0

    def scanned_fun(carry, x):
        chol_a_i, rot_chol_a_i, chol_b_i, rot_chol_b_i = x
        gl_a_i = oe.contract("pj,ji->pi", green_a, chol_a_i, backend="jax")
        gl_b_i = oe.contract("pj,ji->pi", green_b, chol_b_i, backend="jax")
        lt2_green_a_i = oe.contract(
            "pi,ji->pj",
            rot_chol_a_i,
            8 * t2_green_a + 2 * t2_green_ab_a,
            backend="jax",
        )
        lt2_green_b_i = oe.contract(
            "pi,ji->pj",
            rot_chol_b_i,
            8 * t2_green_b + 2 * t2_green_ab_b,
            backend="jax",
        )
        carry[0] += 0.5 * (
            oe.contract("pi,pi->", gl_a_i, lt2_green_a_i, backend="jax")
            + oe.contract("pi,pi->", gl_b_i, lt2_green_b_i, backend="jax")
        )
        glgp_a_i = oe.contract(
            "pi,it->pt", gl_a_i, greenp_a, backend="jax"
        )
        glgp_b_i = oe.contract(
            "pi,it->pt", gl_b_i, greenp_b, backend="jax"
        )
        l2t2_a = 0.5 * oe.contract(
            "pt,qu,ptqu->",
            glgp_a_i,
            glgp_a_i,
            t2_aa,
            backend="jax",
        )
        l2t2_b = 0.5 * oe.contract(
            "pt,qu,ptqu->",
            glgp_b_i,
            glgp_b_i,
            t2_bb,
            backend="jax",
        )
        l2t2_ab = oe.contract(
            "pt,qu,ptqu->",
            glgp_a_i,
            glgp_b_i,
            t2_ab,
            backend="jax",
        )
        carry[1] += l2t2_a + l2t2_b + l2t2_ab
        return carry, 0.0

    [e2_2_2_2, e2_2_3], _ = lax.scan(
        scanned_fun, [0.0, 0.0], (chol_a, rot_chol_a, chol_b, rot_chol_b)
    )
    e2_2_2 = e2_2_2_1 + e2_2_2_2
    e2_2 = e2_2_1 + e2_2_2 + e2_2_3 # <HF|T2 h2|walker>/<HF|walker>

    t = t1g + gt2g # <HF|T1+T2|walker>/<HF|walker>
    e0 = h0 + e1_0 + e2_0 # h0 + <HF|h1+h2|walker>/<HF|walker>
    e1 = e1_1 + e1_2 + e2_1 + e2_2 # <HF|(T1+T2)(h1+h2)|walker>/<HF|walker>

    return jnp.real(t), jnp.real(e0), jnp.real(e1)

In [ ]:
# emf = -3.0682204533536
# lno_eccsd = -0.008423497276034005

In [6]:
walker_up, walker_dn = prop_data['walkers'][0][0], prop_data['walkers'][1][0]
trial._calc_eorb_pt(walker_up,walker_dn,ham_data,wave_data)

(Array(0., dtype=float64),
 Array(-0.008247, dtype=float64),
 Array(0., dtype=float64),
 Array(0., dtype=float64))

In [ ]:
walker_up_r = np.random.randn(*walker_up.shape)
walker_dn_r = np.random.randn(*walker_dn.shape)

0.4091269673929979
0.8149628578785431
0.8149616665566066
